# [문제] Managing Conversation History
- trim_messages()
- RunnablePassthrough.assign
- itemgetter()
- session ID를 사용하여 대화(Multi Turn Conversation)를 진행하세요.
- LLM 모델은 과거 대화를 기억하지 못합니다.

In [ ]:
## 1. 모듈 import
from operator import itemgetter
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.chat_history import InMemoryChatMessageHistory, BaseChatMessageHistory
from langchain_core.messages import SystemMessage, trim_messages
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables.history import RunnableWithMessageHistory

## 2. .env 파일에서 환경변수 읽어오기
load_dotenv()

## 3. 세션별 대화 히스토리를 저장할 임시 메모리 저장소
store = {}

## 4. prompt 생성
messages = [
    ('system', '''당신은 이력서 작성 컨설턴트입니다.
    아래 정보를 바탕으로 지원자 입장에서 2000자 이내로 이력서를 작성합니다.
    문장은 자연스럽고 매끄럽게 작성합니다.'''),
	('placeholder', '{chat_history}'),
	('user', '{query}')
]

prompt = ChatPromptTemplate.from_messages(messages=messages)

## 5. ChatOpenAI 인스턴스 생성: 모델 생성
llm = ChatOpenAI(
    model='gpt-4o',
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

## 6. Trim 설정
trimmer = trim_messages(
    max_tokens=50,
    strategy='last',
    token_counter=llm,
    include_system=True,
    allow_partial=False,
    start_on='human',
)

## 7. chain 구성
chain = (
    RunnablePassthrough.assign(chat_history=itemgetter('chat_history') | trimmer)    
    | prompt 
    | llm
)

## 8. 함수 정의
def get_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

## 9. chain에 대화 히스토리 기능을 래핑해서 추가
with_message_history = RunnableWithMessageHistory(
    chain,
    get_history,
    input_messages_key='query',
    history_messages_key='chat_history'
)

## 10. chain 실행
while True:
    query = input('이력서 작성 컨설턴트입니다. 질문하세요. [종료: S] >>> ')

    if query.upper() == 'S':
        break

    with_message_history.invoke(
        {'query': query},
        config={'configurable': {'session_id': '5678'}}      
    )

[이력서]

**이름:** 김말똥  
**연락처:** 010-8888-8888  
**주소:** 경기도 군포시  

---

**소개**  
안녕하십니까? 저는 건축 내외장 시공분야에서 실력을 쌓아가고 있는 김말똥입니다. 기본에 충실한 시공과 고객만족을 최우선으로 생각하며, 주어진 역할을 성실히 수행하는 것을 목표로 두고 있습니다. 

---

**기술**  
- **미장**  
  다양한 건축 현장에서 공정에 맞게 미장 작업을 수행해왔습니다. 세밀한 손길과 효

In [9]:
for info in get_history('5678').messages:
    print(info.content)
    print('*' * 100)

이름: 김말똥, 나이: 22, 연락처: 010-8888-8888, 주소: 경기도 군포시, 기술: 미장, 도배
****************************************************************************************************
[이력서]

**이름:** 김말똥  
**연락처:** 010-8888-8888  
**주소:** 경기도 군포시  

---

**소개**  
안녕하십니까? 저는 건축 내외장 시공분야에서 실력을 쌓아가고 있는 김말똥입니다. 기본에 충실한 시공과 고객만족을 최우선으로 생각하며, 주어진 역할을 성실히 수행하는 것을 목표로 두고 있습니다. 

---

**기술**  
- **미장**  
  다양한 건축 현장에서 공정에 맞게 미장 작업을 수행해왔습니다. 세밀한 손길과 효
****************************************************************************************************
